In [1]:
import csv
import collections
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample

In [2]:
class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str, default = None
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str], default = None
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool, default = True
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=False, **kwargs):
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, final = False, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        start = time.time()
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...{i / len(self.labels) * 100}%")
            print(f"{(time.time() - start) / 60} minutes")
            if (final):
                predictor.fit(train_data=train_data[train_data[label] > float('-inf')]
                              , tuning_data = tuning_data
                              ,presets = 'best_quality'
                              ,num_bag_folds = 5,num_bag_sets = 2
                              , **kwargs)
            else:
                predictor.fit(train_data=train_data[train_data[label] > float('-inf')]
                              , tuning_data = tuning_data
                              ,presets = 'medium_quality'
                              #,presets = 'best_quality'
                              #,num_bag_folds = 5,num_bag_sets = 2
                              , **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            
            eval_dict[label] = predictor.evaluate(data[data[label] > float('-inf')], **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for i,label in enumerate(self.labels):
            print(f"Predicting with TabularPredictor for label: {label} ...{i / len(self.labels) * 100}%")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [3]:
#1 Load my ccl's ssGSEA signature
myCCLSignature = []
for name in ['sample.c2.cp.biocarta.gct',
             'sample.c2.cp.kegg.gct',
             'sample.c2.cp.pid.gct',
             'sample.c2.cp.reactome.gct',
             'sample.c2.cp.wiki.gct',
             'sample.c6.gct',
             'sample.hallmark.gct']:    
    with open(name, mode ='r')as file:
        csvFile = csv.reader(file)
        CCLSignature = list(csvFile)[2:]
        print(len(CCLSignature))
    for i, row in enumerate(CCLSignature):
        temp = CCLSignature[i][0].split('\t')
        if i > 0:
            CCLSignature[i] = [temp[0]] + [float(d) for d in temp[2:]]
        else:
            CCLSignature[i] = [temp[0]] + temp[2:]
    if not myCCLSignature:
        myCCLSignature += CCLSignature
    else:
        myCCLSignature += CCLSignature[1:]

293
187
197
1616
665
280
53


In [4]:
#2 Load CCLE ssGSEA signature
CCLECCLSignature = []
for name in ['ccle.c2.cp.biocarta.gct',
             'ccle.c2.cp.kegg.gct',
             'ccle.c2.cp.pid.gct',
             'ccle.c2.cp.reactome.gct',
             'ccle.c2.cp.wiki.gct',
             'ccle.c6.gct',
             'ccle.hallmark.gct']:
    with open(name, mode ='r')as file:
        csvFile = csv.reader(file)
        CCLSignature = list(csvFile)[2:]
        print(len(CCLSignature))
    for i, row in enumerate(CCLSignature):
        temp = CCLSignature[i][0].split('\t')
        CCLSignature[i] = [temp[0]] + temp[2:]
    if not CCLECCLSignature:
        CCLECCLSignature += CCLSignature
    else:
        CCLECCLSignature += CCLSignature[1:]

293
187
197
1616
665
280
53


In [5]:
#3 Load CTRP cclName to AUC map
cclToAUCdict = collections.defaultdict(list)
with open('CTRP_CCL_AUC.gct', mode ='r') as file:
    csvFile = csv.reader(file)
    CTRPCCLAUC = list(csvFile)
    CTRPCCLAUC = [''.join(sub).split('\t') for sub in CTRPCCLAUC]
    cclNames = CTRPCCLAUC[3][4:]

for i,cclName in enumerate(cclNames):
    cclToAUCdict[cclName] = [float( '-inf' if sub[4+i] == 'NaN' else sub[4+i]) for sub in CTRPCCLAUC[7:]]

In [6]:
#4 Load ccleID to ctrpName map
CCLEidToCTRPNameDict = collections.defaultdict(str)
CCLEidToDiseaseName = collections.defaultdict(str)
with open('sample_info.csv', mode ='r') as file:
    csvFile = csv.reader(file)
    mapInfos = list(csvFile)
    for mapInfo in mapInfos[1:]:
        CCLEidToCTRPNameDict[mapInfo[0]] = mapInfo[2]  
        CCLEidToDiseaseName[mapInfo[0]] = mapInfo[12]

In [7]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [8]:
# prediction data 
predictData = pd.DataFrame(data = myCCLSignature).transpose()
new_header = predictData.iloc[0] 
predictData = predictData[1:] 
predictData.columns = new_header 
predictData = predictData.apply(pd.to_numeric, errors='ignore')
predictData = predictData.set_index(['Name'])
predictData = normalize(predictData)
predictData.head()

,BIOCARTA_GRANULOCYTES_PATHWAY,BIOCARTA_LYM_PATHWAY,BIOCARTA_BLYMPHOCYTE_PATHWAY,BIOCARTA_CARM_ER_PATHWAY,BIOCARTA_LAIR_PATHWAY,BIOCARTA_VDR_PATHWAY,BIOCARTA_MTA3_PATHWAY,BIOCARTA_GABA_PATHWAY,BIOCARTA_EGFR_SMRTE_PATHWAY,BIOCARTA_MONOCYTE_PATHWAY,...,HALLMARK_COAGULATION,HALLMARK_IL2_STAT5_SIGNALING,HALLMARK_BILE_ACID_METABOLISM,HALLMARK_PEROXISOME,HALLMARK_ALLOGRAFT_REJECTION,HALLMARK_SPERMATOGENESIS,HALLMARK_KRAS_SIGNALING,HALLMARK_KRAS_SIGNALING_UP,HALLMARK_KRAS_SIGNALING_DN,HALLMARK_PANCREAS_BETA_CELLS
Name,,,,,,,,,,,,,,,,,,,,,
THP1_d3_B.TPM,0.991211,0.985279,0.984154,0.876359,0.993570,0.595547,0.475054,0.492397,0.594030,0.986427,...,0.675939,0.809839,0.680037,0.774811,0.928606,0.793392,1.000000,0.880986,0.337452,0.670398
2D_1.TPM,0.160241,0.161473,0.059402,0.379321,0.045364,1.000000,0.089837,0.554072,0.363435,0.187854,...,0.025288,0.065203,0.221749,0.194506,0.019127,0.555587,0.000000,0.037313,0.343351,0.248108
TRId7CWa.quant.TPM,0.438166,0.443539,0.540397,0.000000,0.506852,0.304020,0.583516,0.429940,1.000000,0.700186,...,0.927243,0.736486,0.484722,0.130770,0.436573,0.053547,0.354081,0.771492,0.968649,0.720934
H7.TPM,0.553921,0.562553,0.428064,1.000000,0.724494,0.403809,0.852908,0.493875,0.511236,0.531963,...,0.822794,0.816438,0.113695,0.248351,0.476935,0.189215,0.544488,0.707475,0.643843,0.258149
THP1_d3_A.TPM,0.992471,0.992554,0.973644,0.795964,0.999802,0.548292,0.615737,0.447267,0.474747,0.989311,...,0.691364,0.826434,0.710767,0.877729,0.928190,0.791363,0.947641,0.933168,0.476588,0.819935


In [9]:
# Prepare train set
trainData = pd.DataFrame(data = CCLECCLSignature).transpose()
new_header = trainData.iloc[0] 
trainData = trainData[1:] 
trainData.columns = new_header 

In [10]:
# filter valid ID
validSet = set()
for name in trainData['Name']:
    if CCLEidToCTRPNameDict[name] in cclToAUCdict:
        validSet.add(name)
trainData = trainData.loc[trainData['Name'].isin(validSet)]
trainData = trainData.reset_index(drop = True)
trainData = trainData.set_index(['Name'])
trainData = trainData.apply(pd.to_numeric)
trainData = normalize(trainData)
trainData = trainData[trainData.index.map(lambda x: 'Brain Cancer' in CCLEidToDiseaseName[x])]
trainData.head()

,BIOCARTA_GRANULOCYTES_PATHWAY,BIOCARTA_LYM_PATHWAY,BIOCARTA_BLYMPHOCYTE_PATHWAY,BIOCARTA_CARM_ER_PATHWAY,BIOCARTA_LAIR_PATHWAY,BIOCARTA_VDR_PATHWAY,BIOCARTA_MTA3_PATHWAY,BIOCARTA_GABA_PATHWAY,BIOCARTA_EGFR_SMRTE_PATHWAY,BIOCARTA_MONOCYTE_PATHWAY,...,HALLMARK_COAGULATION,HALLMARK_IL2_STAT5_SIGNALING,HALLMARK_BILE_ACID_METABOLISM,HALLMARK_PEROXISOME,HALLMARK_ALLOGRAFT_REJECTION,HALLMARK_SPERMATOGENESIS,HALLMARK_KRAS_SIGNALING,HALLMARK_KRAS_SIGNALING_UP,HALLMARK_KRAS_SIGNALING_DN,HALLMARK_PANCREAS_BETA_CELLS
Name,,,,,,,,,,,,,,,,,,,,,
ACH-000570,0.110241,0.291122,0.046545,0.411979,0.243086,0.676126,0.511251,0.415892,0.433882,0.374186,...,0.569109,0.544746,0.255964,0.425612,0.222951,0.370397,0.762753,0.687468,0.189627,0.276940
ACH-000075,0.441704,0.498651,0.489649,0.690755,0.501035,0.433048,0.549499,0.535556,0.482125,0.491177,...,0.699880,0.729686,0.201376,0.381000,0.421537,0.462070,0.848547,0.718612,0.092962,0.213359
ACH-000269,0.360292,0.542315,0.357031,0.328689,0.419947,0.623150,0.471118,0.417255,0.473060,0.480172,...,0.538858,0.611223,0.201457,0.380954,0.406162,0.472731,0.637780,0.479822,0.091888,0.158532
ACH-000376,0.240780,0.457245,0.135034,0.803859,0.441461,0.644449,0.590507,0.707571,0.555921,0.496256,...,0.643381,0.631357,0.284282,0.516030,0.386196,0.568442,0.697017,0.553638,0.102042,0.179885
ACH-000368,0.226606,0.501144,0.073657,0.574459,0.459650,0.534463,0.542507,0.503628,0.616213,0.489169,...,0.701306,0.636327,0.134946,0.380434,0.358833,0.510700,0.647769,0.559323,0.192168,0.178395


In [11]:
if not all(trainData.columns == predictData.columns):
    raise Exception("Column do not match!")

In [12]:
#labels data
labelsDataOriginal = pd.DataFrame(columns = [sub[1] for sub in CTRPCCLAUC[7:]])
for name in trainData.index:
    labelsDataOriginal.loc[len(labelsDataOriginal.index)] = cclToAUCdict[CCLEidToCTRPNameDict[name]]
labelsDataOriginal = labelsDataOriginal.set_index(trainData.index)

In [13]:
# labelsDataOriginal.rename(columns={"neuronal differentiation inducer III": "neuronal_differentiation_inducer_III"}, inplace=True)

In [14]:
#add high priority at the begining. e.g dasatinib
labels = list(labelsDataOriginal.columns)

In [15]:
#constants
problem_types = ['regression'] 
eval_metrics = ['mean_squared_error']
time_limit = 60 * 60 * 24
tops = [100,300,600]

In [16]:
visited = set()
for entry in os.scandir(os.getcwd()):
    if entry.is_dir():
        visited.add(entry.name)

In [17]:
# #Comment as it already trained

# for label in labels:
#     if label == "neuronal differentiation inducer III" or label in visited:
#         continue
#     visited.add(label)
#     print(label)
    
#     labelData = labelsDataOriginal[[label]]    
#     trainDataSet = pd.concat([trainData, labelData], axis = 1)
   
#     #first time training 
#     save_path = label + '/' + 'GlioML_BrainCancer_predictDrugAUC_Full_Feature_Medium_Quality_Model_' + label 
#     multi_predictor = MultilabelPredictor(labels=[label], problem_types=problem_types, eval_metrics=eval_metrics, path=save_path)
#     multi_predictor.fit(trainDataSet, time_limit=time_limit)
    
#     result = multi_predictor.predict(predictData)
#     result.to_csv(label + '/' + 'GlioML_predictDrugAUC_Full_Feature_Medium_Quality_Result_' + label+ '.csv')

In [18]:
result = []
for label in labels:
    if label == "neuronal differentiation inducer III":
        continue
    
    save_path = label + '/' + 'GlioML_BrainCancer_predictDrugAUC_Full_Feature_Medium_Quality_Model_' + label 
    predictor = MultilabelPredictor.load(save_path).get_predictor(label)
    scores = predictor.leaderboard()
    scores = scores[['model','score_val']]
    if len(result) ==0:
        result.append([''] + list(scores.sort_values(by='model')['model']))
    result.append([label] + list(scores.sort_values(by='model')['score_val']))

pd.DataFrame(result).to_csv('BrainCancer_MSEscores.csv')

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.076856       0.053000  15.728000                0.000000           0.221001            2       True         12
1              CatBoost  -0.076856       0.053000  15.506999                0.053000          15.506999            1       True          6
2            LightGBMXT  -0.082794       0.012000   1.188119                0.012000           1.188119            1       True          3
3              LightGBM  -0.082794       0.014004   0.872027                0.014004           0.872027            1       True          4
4         LightGBMLarge  -0.088285       0.012001   1.358513                0.012001           1.358513            1       True         11
5       NeuralNetFastAI  -0.092750       0.015999   2.410058                0.015999           2.410058            1       True          8
6       RandomForestMSE  -0

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.216985       0.157001   4.369110                0.000000           0.221004            2       True         12
1        NeuralNetTorch  -0.216985       0.157001   4.148106                0.157001           4.148106            1       True         10
2       RandomForestMSE  -0.263841       0.049000   1.797017                0.049000           1.797017            1       True          5
3              CatBoost  -0.269923       0.054002  41.168410                0.054002          41.168410            1       True          6
4         ExtraTreesMSE  -0.280428       0.048002   1.367054                0.048002           1.367054            1       True          7
5               XGBoost  -0.282868       0.006000   1.640167                0.006000           1.640167            1       True          9
6       NeuralNetFastAI  -0

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -3.046306       0.024999   3.055535                0.000000           0.223001            2       True         12
1       NeuralNetFastAI  -3.047996       0.013001   2.045528                0.013001           2.045528            1       True          8
2            LightGBMXT  -3.090844       0.011997   0.787006                0.011997           0.787006            1       True          3
3              LightGBM  -3.090844       0.013000   0.794553                0.013000           0.794553            1       True          4
4         LightGBMLarge  -3.095682       0.013003   1.263005                0.013003           1.263005            1       True         11
5              CatBoost  -3.121736       0.052999  15.126283                0.052999          15.126283            1       True          6
6        NeuralNetTorch  -3

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.327934       0.019001   3.460054                0.001000           0.219001            2       True         12
1               XGBoost  -0.375897       0.006000   2.288032                0.006000           2.288032            1       True          9
2       NeuralNetFastAI  -0.376141       0.013000   1.956028                0.013000           1.956028            1       True          8
3              LightGBM  -0.385578       0.012000   0.767004                0.012000           0.767004            1       True          4
4            LightGBMXT  -0.385578       0.012001   0.953021                0.012001           0.953021            1       True          3
5              CatBoost  -0.385745       0.052000  15.745676                0.052000          15.745676            1       True          6
6         LightGBMLarge  -0

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -2.505531       0.207005   8.297646                0.000000           0.218001            2       True         12
1        NeuralNetTorch  -2.520130       0.194001   5.895101                0.194001           5.895101            1       True         10
2       NeuralNetFastAI  -2.591173       0.013004   2.184544                0.013004           2.184544            1       True          8
3         LightGBMLarge  -2.872593       0.012000   1.275023                0.012000           1.275023            1       True         11
4               XGBoost  -3.127760       0.194001   2.716470                0.194001           2.716470            1       True          9
5         ExtraTreesMSE  -3.322543       0.033000   1.469029                0.033000           1.469029            1       True          7
6       RandomForestMSE  -3

...

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.504162       0.018000  2.831077                0.001000           0.219002            2       True         12
1               XGBoost  -0.819506       0.005000  1.698035                0.005000           1.698035            1       True          9
2        NeuralNetTorch  -1.419329       0.262004  4.630078                0.262004           4.630078            1       True         10
3         LightGBMLarge  -1.496506       0.012000  0.914041                0.012000           0.914041            1       True         11
4         ExtraTreesMSE  -1.654025       0.049001  1.294029                0.049001           1.294029            1       True          7
5       NeuralNetFastAI  -1.654588       0.012000  1.638572                0.012000           1.638572            1       True          8
6              CatBoost  -1.676365

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.599432       0.236003  17.834377                0.000000           0.218001            2       True         12
1        NeuralNetTorch  -0.599467       0.183001   3.920125                0.183001           3.920125            1       True         10
2              CatBoost  -0.646577       0.053002  13.696250                0.053002          13.696250            1       True          6
3       NeuralNetFastAI  -0.885153       0.012000   1.630559                0.012000           1.630559            1       True          8
4         LightGBMLarge  -0.960655       0.013000   0.919008                0.013000           0.919008            1       True         11
5            LightGBMXT  -0.969723       0.011999   0.765008                0.011999           0.765008            1       True          3
6              LightGBM  -0

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.800955       0.228523   4.667066                0.000000           0.221002            2       True         12
1        NeuralNetTorch  -0.812864       0.216522   3.273053                0.216522           3.273053            1       True         10
2       NeuralNetFastAI  -0.834638       0.013001   1.952042                0.013001           1.952042            1       True          8
3         LightGBMLarge  -0.850255       0.012002   1.173011                0.012002           1.173011            1       True         11
4              LightGBM  -0.854978       0.010999   0.768008                0.010999           0.768008            1       True          4
5            LightGBMXT  -0.854978       0.012000   0.952032                0.012000           0.952032            1       True          3
6              CatBoost  -0

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -1.355561       0.075001   3.999054                0.000000           0.218002            2       True         12
1         LightGBMLarge  -1.368305       0.013001   1.245028                0.013001           1.245028            1       True         11
2        KNeighborsUnif  -1.402125       0.049000   0.415004                0.049000           0.415004            1       True          1
3        KNeighborsDist  -1.406723       0.060001   0.414004                0.060001           0.414004            1       True          2
4       NeuralNetFastAI  -1.545490       0.013000   2.121020                0.013000           2.121020            1       True          8
5              CatBoost  -1.603997       0.053001  26.249479                0.053001          26.249479            1       True          6
6       RandomForestMSE  -1

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -1.699766       0.067999   6.741427                0.001000           0.222001            2       True         12
1       NeuralNetFastAI  -1.774114       0.012000   2.031049                0.012000           2.031049            1       True          8
2        NeuralNetTorch  -1.885620       0.114001   3.807038                0.114001           3.807038            1       True         10
3         LightGBMLarge  -1.971146       0.012000   1.290009                0.012000           1.290009            1       True         11
4       RandomForestMSE  -2.057312       0.049001   2.411342                0.049001           2.411342            1       True          5
5         ExtraTreesMSE  -2.077220       0.049000   1.499010                0.049000           1.499010            1       True          7
6               XGBoost  -2